In [1]:
import tensorflow as tf
import random
import numpy as np
from common_functions import *

In [2]:
sess = tf.Session()
Input = tf.placeholder(tf.float32, [None, 2])
Labels = tf.placeholder(tf.float32, [None, 2])
Money_Line = tf.placeholder(tf.float32, [None, 2])

worth_0 = tf.layers.dense(inputs=Input, units = 256, activation=tf.nn.relu)
worth_1 = tf.layers.dense(inputs=worth_0, units = 256, activation=tf.nn.relu)
worth_2 = tf.layers.dense(inputs=worth_1, units= 256, activation=tf.nn.relu)

output = tf.layers.dense(inputs=worth_2, units=2, activation=tf.nn.softmax)
scale = tf.layers.dense(inputs=worth_2, units=1, activation = tf.nn.sigmoid)

profit = tf.reduce_sum(Labels* Money_Line*output*scale, axis=1) - scale
avg_profit = tf.reduce_mean(profit)
loss = -avg_profit 

opt = tf.train.AdamOptimizer(3e-5).minimize(loss)

sess.run(tf.global_variables_initializer())


In [3]:
with open("odds_example_soccer.csv", "r") as odds:
    o = odds.read()
    o = o.split("\n")[3:]
    
for i,x in enumerate(o):
    o[i] = o[i].split(",")



In [4]:
extracted = [extract_features_and_labels(i,18,19 ,31, 2) for i in o]
extracted = list(filter(lambda x: x != -1, extracted ))

random.shuffle(extracted)
cutoff = int(len(extracted)*0.9)

train = extracted[:cutoff]
#train = train+ [augment(x) for x in train]
test = extracted[cutoff:]

In [6]:
for i in range(100000):
    features, labels = pack_sample(train, 128)
    a_p, L, __ = sess.run([avg_profit, loss, opt], feed_dict = {Input:features, Labels:labels, Money_Line: features})
    
    if i % 1000 == 0:
        print("Iteration: "+ str(i))
        print("Training loss: "+ str(L))
        print("Training average profit: "+ str(a_p))
        features, labels= pack_sample(test) 
        scales, avg_prof, prof, out, L = sess.run([scale, avg_profit, profit, output, loss], feed_dict = {Input:features, Labels:labels, Money_Line: features})
        print("Testing loss: "+ str(L))
        print("Testing average profit: "+ str(avg_prof))
        print("Scales: " + str(scales[:5]))
        
  #      print("Outputs: "+ str(out[:5]))
  #      print("Inputs: "+ str(features[:5]))
  #      print("Labels: "+ str(labels[:5]))
  #      print("Profits"+ str(prof[:5]))

Iteration: 0
Training loss: -0.08205218
Training average profit: 0.08205218
Testing loss: -0.015583966
Testing average profit: 0.015583966
Scales: [[9.999634e-01]
 [2.492629e-06]
 [9.999994e-01]
 [9.031086e-01]
 [1.000000e+00]]
Iteration: 1000
Training loss: -0.071399
Training average profit: 0.071399
Testing loss: -0.09080936
Testing average profit: 0.09080936
Scales: [[1.000000e+00]
 [1.000000e+00]
 [9.998684e-01]
 [5.663958e-07]
 [9.835934e-01]]
Iteration: 2000
Training loss: -0.010317154
Training average profit: 0.010317154
Testing loss: -0.17861241
Testing average profit: 0.17861241
Scales: [[0.01470068]
 [0.9990721 ]
 [1.        ]
 [1.        ]
 [1.        ]]
Iteration: 3000
Training loss: -0.08959471
Training average profit: 0.08959471
Testing loss: -0.081671424
Testing average profit: 0.081671424
Scales: [[0.9996743]
 [1.       ]
 [0.9999794]
 [0.9999999]
 [1.       ]]
Iteration: 4000
Training loss: 0.014359862
Training average profit: -0.014359862
Testing loss: -0.008373484
Te

Iteration: 36000
Training loss: 0.027286293
Training average profit: -0.027286293
Testing loss: -0.07353329
Testing average profit: 0.07353329
Scales: [[6.1123302e-08]
 [9.9987686e-01]
 [1.0000000e+00]
 [9.9999988e-01]
 [2.0331336e-06]]
Iteration: 37000
Training loss: 0.051375374
Training average profit: -0.051375374
Testing loss: -0.0027433578
Testing average profit: 0.0027433578
Scales: [[1.        ]
 [0.9995906 ]
 [0.99942905]
 [0.9999826 ]
 [0.99745363]]
Iteration: 38000
Training loss: -0.031099211
Training average profit: 0.031099211
Testing loss: -0.16390054
Testing average profit: 0.16390054
Scales: [[1.       ]
 [1.       ]
 [0.9998266]
 [1.       ]
 [1.       ]]
Iteration: 39000
Training loss: 0.019606467
Training average profit: -0.019606467
Testing loss: -0.17223105
Testing average profit: 0.17223105
Scales: [[0.99999964]
 [1.        ]
 [1.        ]
 [0.9999943 ]
 [0.0363136 ]]
Iteration: 40000
Training loss: -0.11948632
Training average profit: 0.11948632
Testing loss: -0.0

Iteration: 72000
Training loss: -0.17251793
Training average profit: 0.17251793
Testing loss: -0.10387822
Testing average profit: 0.10387822
Scales: [[9.9999833e-01]
 [7.4678339e-02]
 [3.5054298e-05]
 [9.9999893e-01]
 [1.0000000e+00]]
Iteration: 73000
Training loss: -0.1546405
Training average profit: 0.1546405
Testing loss: 0.20082095
Testing average profit: -0.20082095
Scales: [[1.0000000e+00]
 [9.9945515e-01]
 [1.0517232e-08]
 [9.9971241e-01]
 [6.4104368e-11]]
Iteration: 74000
Training loss: -0.037432678
Training average profit: 0.037432678
Testing loss: -0.1016606
Testing average profit: 0.1016606
Scales: [[1.0000000e+00]
 [1.0000000e+00]
 [7.8589743e-01]
 [1.9699241e-04]
 [4.2082135e-07]]
Iteration: 75000
Training loss: 0.01051667
Training average profit: -0.01051667
Testing loss: -0.06605356
Testing average profit: 0.06605356
Scales: [[9.9998927e-01]
 [3.3486133e-10]
 [2.5789373e-09]
 [5.7180133e-12]
 [9.9999976e-01]]
Iteration: 76000
Training loss: -0.0130671365
Training average

In [14]:
feat = [x[0] for x in train[50000:75000]] #+test
lab = [x[1] for x in train[50000:75000]] #+testtrain[:25000]
a_p = sess.run(avg_profit, feed_dict={Input:feat, Labels:lab, Money_Line: feat})

print(a_p)

0.068650216
